In [1]:
import re
import json
import time
import hashlib
import requests
import warnings
import pandas as pd
from PIL import Image
from io import BytesIO
from IPython.display import display
warnings.filterwarnings("ignore")

In [2]:
class VpnLogin(object):
    def __init__(self, vpn_name='vpns_username', vpn_passwd='vpns_passwd', uims_name='uims_username', uims_passwd='uims_passwd'):
        self.vpn_name = vpn_name    # vpns的用户名
        self.vpn_passwd = vpn_passwd    # vpns的密码
        self.uims_name = uims_name    # uims的用户名
        self.uims_passwd = uims_passwd    # uims的密码
        self.pattern1 = re.compile(r'用户名密码错误')   # 判断是否登vpns成功的正则表达式
        self.pattern2 = re.compile(r'<span class="error_message" id="error_message">(.*)</span>')   # 判断是否登uims成功的正则表达式
        self._init_session()
        
    def _init_user_vpns(self):
        print('请输入vpns的用户名：', end='')
        self.vpn_name = input()
        print('请输入vpns的密码：', end='')
        self.vpn_passwd = input()
        
    def _init_user_uims(self):
        print('请输入uims的用户名：', end='')
        self.uims_name = input()
        print('请输入uims的密码：', end='')
        self.uims_passwd = input()
        
    def _init_session(self):
        session = requests.session()
        session.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',   
            'Content-Type': 'application/x-www-form-urlencoded',  # 以form形式提交表单
            'Connection': 'keep-alive',
        }
        self.session = session
    
    def _get_md5_passwd(self):
        # 通过分析js可得
        b_password = 'UIMS' + self.uims_name + self.uims_passwd
        md5_password = hashlib.md5(b_password.encode('UTF-8')).hexdigest()
        return md5_password
    
    def _login_vpns(self):
        # 获取wengine_vpn_ticket
        url1 = 'https://vpns.jlu.edu.cn/'
        self.session.post(url1, verify=False)
        # 登陆vpns.jlu.edu.cn
        url_2 = 'https://vpns.jlu.edu.cn/do-login'    
        url2_data = 'auth_type=local&username=' + self.vpn_name + '&sms_code=&password=' + self.vpn_passwd
        verify_result = self.session.post(url_2, data=url2_data, verify=False)
        verify_res = re.findall(self.pattern1, verify_result.content.decode())
        return verify_res
    
    def _get_captcha(self):
        captcha_html = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/open/get-captcha-image.do?vpn-1&s=1'
        captcha_img = self.session.get(captcha_html, verify = False)
        # 保存验证码到本地并打开
        with open('./captcha.jpg', 'wb+') as f:
            f.write(captcha_img.content)
            f.close()
        image = Image.open('./captcha.jpg')    # read image
        display(image)
        print('请输入验证码：', end='')
        vcode = input()
        return vcode
    
    def _login_uims(self):
        verify_html = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/j_spring_security_check'
        verify_dict = {  
            'username': self.uims_name,
            'password': self._get_md5_passwd(), 
            'mousePath': '',
            'vcode': self._get_captcha()
        }
        verify_result = self.session.post(verify_html, data = verify_dict, allow_redirects=True, verify=False)
        verify_res = re.findall(self.pattern2, verify_result.content.decode())
        return verify_res
    
    def login(self):
        # 登录vpns
        vpns_res = self._login_vpns()
        while len(vpns_res):
            print('vpns登录失败，原因：' + vpns_res[0])
            self._init_user_vpns()    # vpns账号密码输入错误，重新输入
            self._init_session()
            vpns_res = self._login_vpns()
        print('vpns登录成功')
        # 登录uims
        uims_res = self._login_uims()
        while len(uims_res):
            print('uims登录失败，原因：' + uims_res[0])
            if(uims_res[0] == '登录错误：验证码无效或过期'):
                uims_res = self._login_uims()    # 验证码无效或过期，只需要重新输入验证码
            else:
                self._init_user_uims()    # uims账号密码错误，需要重新输入信息
                uims_res = self._login_uims()    
        print('uims登录成功')
        
    
    def get_cookies(self):
        cookie = ''
        for x in self.session.cookies:
            cookie += x.name + '=' + x.value + ';'
        cookie = cookie[:-1]
        return cookie

In [3]:
vpn_login = VpnLogin('vpns的用户名', 'vpns的密码', 'uims的用户名', 'uims的密码')

In [4]:
vpn_login.login()

vpns登录成功


请输入验证码：9018
uims登录成功


In [5]:
cookie = vpn_login.get_cookies()
print(cookie)

refresh=1;wengine_vpn_ticket=94ebe2b4fe9d1b94
